In [ ]:
import numpy as np 
import pandas as pd 
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

In [ ]:
#Functions to load the data
def get_data_vote_description():
    df = pd.read_csv('/kaggle/input/french-national-assembly-open-data/df_vote_description.csv')
    df['year'] = df['date'].astype(str).str[0:4]
    df['month'] = df['date'].astype(str).str[5:7]
    df['day'] = df['date'].astype(str).str[8:10]
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day']], errors = 'coerce')
    df['percentage of votes in favor'] = 100*df['pour']/df['nb votants']
    df['accepted'] = 'no'
    df.loc[df['pour'] >= df['requis'], 'accepted'] = 'yes'
    df = df.drop(columns=['date'])
    df.columns = df.columns.str.replace('demandeur ', '')
    return df

def get_data_political_parties():
    df = pd.read_csv('/kaggle/input/french-national-assembly-open-data/df_political_parties.csv')
    df = df.drop(columns=['code'])
    return df

def get_data_deputies():
    df = pd.read_csv('/kaggle/input/french-national-assembly-open-data/df_deputies.csv')
    #create the value age from the date of birth of the deputies
    df['age']  = df['date of birth'].astype(str).str[0:4]
    df['age']  = date.today().year - df['age'].astype(int)
    df['departement'] = df['dep'].astype(str) + ' ('+ df['num_dep'] + ')'
    df['full_name'] = df['first name'].astype(str) + ' '+ df['family name']
    df['title'] = 'Mr.'
    df.loc[df['sex']=='female', 'title'] = 'Mme.'
    return df


def get_data_organs():
    df = pd.read_csv('/kaggle/input/french-national-assembly-open-data/df_organs.csv')
    return df

def get_data_deputies_in_organs():
    df = pd.read_csv('/kaggle/input/french-national-assembly-open-data/df_deputies_missions.csv')
    return df

def get_data_votes():
    df = pd.read_csv('/kaggle/input/french-national-assembly-open-data/df_votes.csv', 
    dtype={
        'pour': float,
        'contre': float,
        'non votants' : float,
        'abstentions' : float,
        'par delegation' : float
            })
    df['vote'] = 1
    df['cause'] = df['cause'].fillna('none')
    df['cause'] = df['cause'].astype("category")
    df['deputy code'] = df['deputy code'].astype("category")
    df['scrutin'] = df['scrutin'].astype("category")
    return df

## Let's do some simple data visualization to start

In [ ]:
df_deputies = get_data_deputies()
df_deputies

Seems like the loading function is working. <br>
Let's load the rest of the data

In [ ]:
df_vote_description = get_data_vote_description()
df_political_parties = get_data_political_parties()
df_organs = get_data_organs()
df_deputies_missions = get_data_deputies_in_organs()
df_votes = get_data_votes()

In [ ]:
df = pd.merge(pd.DataFrame(df_deputies['pol party'].value_counts()), df_political_parties, left_index=True, right_on='abreviated_name')
colors = df['color'].tolist()

fig, ax = plt.subplots(figsize=(7, 7))
ax.pie(df_deputies['pol party'].value_counts(), labels=(df_deputies['pol party'].value_counts().index + ' (' + df_deputies['pol party'].value_counts().map(str) + ')'), 
    wedgeprops = { 'linewidth' : 7, 'edgecolor' : 'white' }, colors=colors)
ax.set_title("Political parties in the french national assembly")
p = plt.gcf()
p.gca().add_artist(plt.Circle( (0,0), 0.7, color='white'))
plt.show()